In [1]:
import numpy as np
import os
import pandas as pd
from ASR.infer_datalayers import AudioInferDataLayer
from ASR.JasperModels import JasperInference
from ruamel.yaml import YAML
import pesq
import librosa
from nemo_asr.helpers import post_process_predictions, word_error_rate
import tqdm

Unable to import APEX. Mixed precision and distributed training will not work.


In [2]:
df = pd.read_csv('/Users/jberkowitz/Datasets/VOiCES_devkit/references/test_index.csv')
dataset_root = '/Users/jberkowitz/Datasets/VOiCES_devkit/'
df = df[df['source_length']==df['noisy_length']]
print(len(df))
df.head()

6400


,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,source,speaker,transcript,noisy_length,noisy_sr,noisy_time,source_length,source_sr,source_time
0,0,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940
1,1,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940
2,2,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125
3,3,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125
4,4,130550,10,babb,distant-16k/speech/test/rm4/babb/sp1447/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...,rm4,26,source-16k/test/sp1447/Lab41-SRI-VOiCES-src-sp...,1447,he was inexhaustibly voluble,58720,16000,3.670,58720,16000,3.670


In [3]:
MODEL_YAML = '/Users/jberkowitz/NeMo/examples/asr/configs/quartznet15x5.yaml'
yaml = YAML(typ="safe")
with open(MODEL_YAML) as f:
    model_definition = yaml.load(f)
vocab = model_definition['labels']
model_definition.keys()

dict_keys(['model', 'sample_rate', 'AudioToTextDataLayer', 'AudioToMelSpectrogramPreprocessor', 'SpectrogramAugmentation', 'JasperEncoder', 'labels'])

In [4]:
#pre-trained weights
#encoder_path = '/Users/jberkowitz/Datasets/quartznet_checkpoints/JasperEncoder-STEP-247400.pt'
#decoder_path = '/Users/jberkowitz/Datasets/quartznet_checkpoints/JasperDecoderForCTC-STEP-247400.pt'

#fine-tuned
encoder_path = '/Users/jberkowitz/Datasets/QuartzNet-lr_0.0001-bs_16-e_3-wd_0.001-opt_novograd/checkpoints/JasperEncoder-STEP-2190.pt'
decoder_path = '/Users/jberkowitz/Datasets/QuartzNet-lr_0.0001-bs_16-e_3-wd_0.001-opt_novograd/checkpoints/JasperDecoderForCTC-STEP-2190.pt'

In [5]:
jasper = JasperInference(model_definition)

2020-03-29 13:53:23,020 - INFO - PADDING: 16
2020-03-29 13:53:23,021 - INFO - STFT using conv


In [6]:
jasper.restore_weights(encoder_weight_path=encoder_path,decoder_weight_path=decoder_path)

In [7]:
def process_item(item,dataset_root,jasper_model,sample_rate=16000):
    ground_truth = [item['transcript']]
    result_dict = {'query_name':item['query_name']}
    result_dict['ground_truth']=item['transcript']
    
    noisy_filepath = os.path.join(dataset_root,item['filename'])
    clean_filepath = os.path.join(dataset_root,item['source'])
    
    noisy_waveform,_ = librosa.load(noisy_filepath,sr=sample_rate)
    clean_waveform,_ = librosa.load(clean_filepath,sr=sample_rate)
    
    pesq_nb = pesq.pesq(16000,clean_waveform,noisy_waveform,'nb')
    pesq_wb = pesq.pesq(16000,clean_waveform,noisy_waveform,'wb')
    result_dict['pesq nb'] = pesq_nb
    result_dict['pesq wb'] = pesq_wb
    
    noisy_result = jasper_model.infer(waveforms=[noisy_waveform])
    clean_result = jasper_model.infer(waveforms=[clean_waveform])
    
    result_dict['noisy transcript'] = noisy_result['greedy transcript'][0]
    result_dict['clean transcript'] = clean_result['greedy transcript'][0]
    result_dict['noisy wer'] = word_error_rate([result_dict['noisy transcript']],[result_dict['ground_truth']])
    result_dict['clean wer'] = word_error_rate([result_dict['clean transcript']],[result_dict['ground_truth']])
    
    return result_dict

In [8]:
index = 0
item = df.iloc[index]
test_result = process_item(item,dataset_root,jasper)

In [9]:
%timeit process_item(item,dataset_root,jasper)

1.68 s ± 25.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
result_list = []
for i in tqdm.tqdm(range(len(df))):
    item = df.iloc[i]
    result = process_item(item,dataset_root,jasper)
    result_list.append(result)
result_df = pd.DataFrame(result_list)
result_df.to_csv('finetuned_quartznet.csv')

100%|██████████| 6400/6400 [3:21:35<00:00,  1.62s/it]  


In [10]:
result_df

,clean transcript,clean wer,ground_truth,noisy transcript,noisy wer,pesq nb,pesq wb,query_name
0,and i got on with the lesson very badly i am s...,0.022222,and i got on with the lesson very badly i am s...,and i don't on with the lesson very god i'm su...,0.155556,1.398316,1.120348,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...
1,and i got on with the lesson very badly i am s...,0.022222,and i got on with the lesson very badly i am s...,o not go on i te les an i'm srue o ant verry...,0.977778,1.169913,1.051034,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...
2,and ran up stairs to put away the host of litt...,0.057692,and ran upstairs to put away the host of littl...,e i an upstairs that e lay the rostere regarde...,0.961538,1.211919,1.050670,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...
3,and ran up stairs to put away the host of litt...,0.057692,and ran upstairs to put away the host of littl...,and ran upstairs to put away the host of littl...,0.057692,1.808863,1.317633,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...
4,he was inexhaustibly voluble,0.000000,he was inexhaustibly voluble,he was inexhausticly voluble,0.250000,1.929595,1.321101,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...
5,he was inexhaustibly voluble,0.000000,he was inexhaustibly voluble,e ors ten sal toma all the tay ina,2.250000,1.116900,1.043152,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...
6,it has seldom been surpassed it imposes no con...,0.000000,it has seldom been surpassed it imposes no con...,it has seldom been surpassed it inposes no con...,0.181818,1.827871,1.190343,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130551-sg00...
7,it has seldom been surpassed it imposes no con...,0.000000,it has seldom been surpassed it imposes no con...,it has sal redul or a a toe ut inhar tols is ...,1.318182,1.303290,1.063059,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130551-sg00...
8,the thing that bled and ran screaming and sobb...,0.027778,the thing that bled and ran screaming and sobb...,was the thing that bled and ran screaming and ...,0.250000,2.025725,1.514220,Lab41-SRI-VOiCES-rm4-babb-sp0718-ch129594-sg00...
9,he had an idea i believe that to gabble about ...,0.037037,he had an idea i believe that to gabble about ...,an a e o eis caria o a on a ta e all e re o ...,0.962963,1.303254,1.100150,Lab41-SRI-VOiCES-rm4-babb-sp0718-ch129597-sg00...


In [11]:
len(df['source'].unique())

200

In [12]:
200*2*4*4

6400